<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/HTLim/NLP_Luhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 3.8MB 48.2MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [165]:
# import packages
import re
import string
from konlpy.tag import Hannanum
from konlpy.utils import pprint
import numpy as np

In [223]:
# Luhn class

class Luhn():

    def __init__(self):
        self.sentence_token = None
        self.sig_words = None
        self.sen_sig = []
        self.order_sen = []

    # 텍스트 전처리
    def pre_processor(self, doc):
        seperator = '\n|\.'
        spliter = re.split(seperator, doc)
        
        result = [sentence.strip().replace(",","").replace("‘","").replace("’","").replace("  "," ") for sentence in spliter]
        result = list(filter(None, result))
        result = '. '.join(result)

        return result
    
    # 문장 분리
    def split_sentence(self, pre_process):
        self.sentence_token = pre_process.split('. ')
        return self.sentence_token

    # 토크나이저
    def tokenizer(self, join_sentence):
        h = Hannanum()
        pp = h.pos(join_sentence)
        return pp
    
    # 중요단어 추출
    def significant_word(self, sentence_token):

        sig_words = []
        noun_words = []
        join_sentence = ' '.join(sentence_token)
        pp = self.tokenizer(join_sentence)
        for word in pp:
            # 명사만 추출
            if word[1] == 'N':
                noun_words.append(word[0])
        
        # 문장길이
        sen_len = len(self.sentence_token) 

        # 빈도 기반 중요단어 결정       
        for word in noun_words:
            count = 0
            for sentence in self.sentence_token:
                if word in sentence:
                    count += 1
            if (count/sen_len <= 0.5) and (count/sen_len >= 0.1):
                sig_words.append(word)
        self.sig_words = sig_words
        return sig_words

    # 문장 중요도 계산
    def cal_sentence_sig(self):
        self.sen_sig = []
        for sentence in self.sentence_token:
            tokens = sentence.split()
            sig_data = []
            for i, token in enumerate(tokens):
                if token in self.sig_words:
                    sig_data.append(i)

            # 문서가 작아 윈도우 사이즈 >= 2 이상 & 단어 빈도 0.1 ~ 0.5 사이로 측정
            if len(sig_data)>1:
                window = max(sig_data)-min(sig_data)+1
                sig_score = len(sig_data)/window
            else:
                sig_score = 0
            
            self.sen_sig.append(sig_score)
        return self.sen_sig

    # 중요 문장 순 정렬
    def sentence_ordering(self):
        order = np.argsort(self.sen_sig)[::-1]
        
        for num in order:
            self.order_sen.append(self.sentence_token[num])
        
        return self.order_sen

In [224]:
doc = '''
코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성, 기술‧응용 공동 연구
지난 8일, 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력

코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 ‘코로나19 감염 대응을 위한 의료 인공지능 컨소시엄’이 9월 1일에 출범한다. 

연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.
 
컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학), 김휘영(방사선의과학연구소), 심규원(신경외과), 염준섭(감염내과), 최병욱(영상의학과, 사진), 허진(영상의학과) 교수가 참여했다.

대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.

이 밖에도 전종홍(한국전자통신연구원), 김경훈(정보통신정책연구원), 예종철(한국과학기술원) 교수도 포함됐다.
 
인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고, 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다.

컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회, 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다.
 
한편 오는 7월 24일(금)에는 온라인 워크숍을 개최해 코로나19 감염대응 의료 인공지능 관련 동향발표 및 연구개발 사례 소개가 있을 예정이다.

'''

In [225]:
lu = Luhn()

In [226]:
# 기사 전처리
processed = lu.pre_processor(doc)
processed

'코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성 기술‧응용 공동 연구. 지난 8일 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력. 코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 코로나19 감염 대응을 위한 의료 인공지능 컨소시엄이 9월 1일에 출범한다. 연세대 의대 한국전자통신원 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집 공유 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다. 컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학) 김휘영(방사선의과학연구소) 심규원(신경외과) 염준섭(감염내과) 최병욱(영상의학과 사진) 허진(영상의학과) 교수가 참여했다. 대학군에서는 권인호(동아대) 김남국(울산대) 김윤현(전남대) 김진영(계명대) 박상준(서울대) 신수용(성균관대) 정명진(성균관대) 진광남(서울대) 홍헬렌(서울여대) 교수가 산업군에서는 김기환(루닛) 이동훈(카카오) 정규환(뷰노) 최우식(딥노이드) 최정필(코어라인소프트) 하정우(네이버)가 기업을 대표해 동참했다. 이 밖에도 전종홍(한국전자통신연구원) 김경훈(정보통신정책연구원) 예종철(한국과학기술원) 교수도 포함됐다. 인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다. 컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다. 한편 오는 

In [227]:
# 기사 문장단위 split
ss = lu.split_sentence(processed)
ss

['코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성 기술‧응용 공동 연구',
 '지난 8일 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력',
 '코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 코로나19 감염 대응을 위한 의료 인공지능 컨소시엄이 9월 1일에 출범한다',
 '연세대 의대 한국전자통신원 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발 검증할 예정이다',
 '또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집 공유 활용한다',
 '나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다',
 '컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다',
 '연세대 의과대학에서는 박유랑(의생명시스템정보학) 김휘영(방사선의과학연구소) 심규원(신경외과) 염준섭(감염내과) 최병욱(영상의학과 사진) 허진(영상의학과) 교수가 참여했다',
 '대학군에서는 권인호(동아대) 김남국(울산대) 김윤현(전남대) 김진영(계명대) 박상준(서울대) 신수용(성균관대) 정명진(성균관대) 진광남(서울대) 홍헬렌(서울여대) 교수가 산업군에서는 김기환(루닛) 이동훈(카카오) 정규환(뷰노) 최우식(딥노이드) 최정필(코어라인소프트) 하정우(네이버)가 기업을 대표해 동참했다',
 '이 밖에도 전종홍(한국전자통신연구원) 김경훈(정보통신정책연구원) 예종철(한국과학기술원) 교수도 포함됐다',
 '인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다',
 '컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다',
 '아울러 대한의료인공지능학회 대한영

In [228]:
# 중요단어 추출
tn = lu.significant_word(ss)
print(tn)

['코로나19', '대응', '의료', '인공지능', '컨소시엄', '출범', '산학연병', '인공지능', '전문', '20', '명', '공동', '연구', '8일', '19', '컨소시엄', '협력', '코로나19', '감염', '대응', '관련', '전문가들', '코로나19', '감염', '대응', '의료', '인공지능', '컨소시엄', '출범', '연세대', '의대', '네이버', '등', '산학연병', '20', '명', '참여', '컨소시엄', '인공지능', '기술', '응용', '공동', '연구', '개발', '예정', '인공지능', '기술', '산학연병', '전문가들', '협력', '공동', '대응', '예정', '컨소시엄', '20', '명', '전문가들', '참여', '연세대', '교수', '참여', '교수', '교수', '인공지능', '컨소시엄', '7월', '8일', '창립', '준비', '모임', '연세대', '의대', '영상의학', '최병욱', '컨소시엄', '19', '컨소시엄', '19', '등', '협력', '계획', '코로나19', '대응', '개발', '등', '국내', '관련', '학회', '협력', '계획', '7월', '코로나19', '의료', '인공지능', '관련', '예정']


In [229]:
sen_score = lu.cal_sentence_sig()
sen_score

[0.6666666666666666,
 0.3333333333333333,
 0.4117647058823529,
 0.47058823529411764,
 0,
 0,
 0,
 0,
 0,
 0,
 0.5,
 0.2,
 1.0,
 0.5]

In [230]:
order = lu.sentence_ordering()
order

['아울러 대한의료인공지능학회 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다',
 '코로나19 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성 기술‧응용 공동 연구',
 '한편 오는 7월 24일(금)에는 온라인 워크숍을 개최해 코로나19 감염대응 의료 인공지능 관련 동향발표 및 연구개발 사례 소개가 있을 예정이다',
 '인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다',
 '연세대 의대 한국전자통신원 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발 검증할 예정이다',
 '코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 코로나19 감염 대응을 위한 의료 인공지능 컨소시엄이 9월 1일에 출범한다',
 '지난 8일 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력',
 '컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다',
 '이 밖에도 전종홍(한국전자통신연구원) 김경훈(정보통신정책연구원) 예종철(한국과학기술원) 교수도 포함됐다',
 '대학군에서는 권인호(동아대) 김남국(울산대) 김윤현(전남대) 김진영(계명대) 박상준(서울대) 신수용(성균관대) 정명진(성균관대) 진광남(서울대) 홍헬렌(서울여대) 교수가 산업군에서는 김기환(루닛) 이동훈(카카오) 정규환(뷰노) 최우식(딥노이드) 최정필(코어라인소프트) 하정우(네이버)가 기업을 대표해 동참했다',
 '연세대 의과대학에서는 박유랑(의생명시스템정보학) 김휘영(방사선의과학연구소) 심규원(신경외과) 염준섭(감염내과) 최병욱(영상의학과 사진) 허진(영상의학과) 교수가 참여했다',
 '컨소시엄에는 15개 기관의 